In [35]:
import sys
sys.path.append('..')

In [36]:
import numpy as np
import pandas as pd
import pickle

import neptune.new as neptune
from neptune.new.integrations.tensorflow_keras import NeptuneCallback

from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from keras.optimizers import RMSprop, Adam, Adamax

from lib.read_data import read_and_join_output_file
from lib.create_pipeline import create_transformation_pipeline
from lib.transform_impute import convert_back_df
from lib.split_data import train_test_group_time_split

In [37]:
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [38]:
# During experiment we can try to use neptune.ai to log all the Tensorflow experiments results
neptune_key = pickle.load(open("./neptune.pkl", "rb"))


## Preparing the Dataset
The train and test sets are split by Township-Ranges, i.e. some Township-Ranges data are either fully in the train or test set.
The target value is the value of that variable for 2021
Thus train/test sets are of shape (number of Township-Ranges, 7 years (2014-2020), the number of features).
The input of 1 data point in the model is of shape (7x81


In [39]:
RANDOM_SEED = 42
# Load the data from the ETL output files
X = read_and_join_output_file()
#X["WELL_COUNT"] = X["WELL_COUNT_PUBLIC"] + X["WELL_COUNT_AGRICULTURE"] + X["WELL_COUNT_DOMESTIC"] + X["WELL_COUNT_INDUSTRIAL"]
#X.drop(columns=["WELL_COUNT_PUBLIC", "WELL_COUNT_AGRICULTURE", "WELL_COUNT_DOMESTIC", "WELL_COUNT_INDUSTRIAL"], inplace=True)
# Split the data into a training and a test set
X_train_df, X_test_df, y_train_df, y_test_df = train_test_group_time_split(X, index=["TOWNSHIP_RANGE", "YEAR"], group="TOWNSHIP_RANGE", random_seed=RANDOM_SEED)
# Create, fit and apply the data imputation pipeline to the training and test sets
impute_pipeline = create_transformation_pipeline(X_train_df, scaler = MinMaxScaler())
X_train_impute = impute_pipeline.fit_transform(X_train_df)
X_test_impute = impute_pipeline.fit_transform(X_test_df)
# Convert the X_train and X_test back to dataframes
X_train_impute_df = convert_back_df(X_train_impute, impute_pipeline, X_train_df)
X_test_impute_df = convert_back_df(X_test_impute, impute_pipeline, X_test_df)
# Keep only the GSE_GWE variable as the outcome variable
scaler = MinMaxScaler()
y_train = scaler.fit_transform(y_train_df[["GSE_GWE"]])
y_test = scaler.transform(y_test_df[["GSE_GWE"]])
X_train_impute_df

TOTALDRILLDEPTH_AVG  WELLYIELD_AVG  STATICWATERLEVEL_AVG  \
TOWNSHIP_RANGE YEAR                                                             
T01N R03E      2014             0.097778       0.018246              0.037145   
               2015             0.095238       0.021053              0.025042   
               2016             0.114286       0.007916              0.022398   
               2017             0.000000       0.013684              0.030885   
               2018             0.083873       0.002474              0.034558   
...                                  ...            ...                   ...   
T32S R30E      2016             0.000000       0.000000              0.000000   
               2017             0.000000       0.000000              0.000000   
               2018             0.000000       0.000000              0.000000   
               2019             0.000000       0.000000              0.000000   
               2020             0.000000       0.000000              0.000000   

                     TOPOFPERFORATEDINTERVAL_AVG  \
TOWNSHIP_RANGE YEAR                                
T01N R03E      2014                     0.098039   
               2015                     0.117647   
               2016                     0.152614   
               2017                     0.127451   
               2018                     0.148257   
...                                          ...   
T32S R30E      2016                     0.000000   
               2017                     0.000000   
               2018                     0.000000   
               2019                     0.000000   
               2020                     0.000000   

                     BOTTOMOFPERFORATEDINTERVAL_AVG  TOTALCOMPLETEDDEPTH_AVG  \
TOWNSHIP_RANGE YEAR                                                            
T01N R03E      2014                        0.111111                 0.105856   
               2015                        0.080460                 0.079848   
               2016                        0.103768                 0.104880   
               2017                        0.082375                 0.081749   
               2018                        0.093934                 0.107605   
...                                             ...                      ...   
T32S R30E      2016                        0.000000                 0.000000   
               2017                        0.000000                 0.000000   
               2018                        0.000000                 0.000000   
               2019                        0.000000                 0.000000   
               2020                        0.000000                 0.000000   

                     VEGETATION_BLUE_OAK-GRAY_PINE  \
TOWNSHIP_RANGE YEAR                                  
T01N R03E      2014                       0.000037   
               2015                       0.000037   
               2016                       0.000037   
               2017                       0.000037   
               2018                       0.000037   
...                                            ...   
T32S R30E      2016                       0.033178   
               2017                       0.033178   
               2018                       0.033178   
               2019                       0.033178   
               2020                       0.033178   

                     VEGETATION_CALIFORNIA_COAST_LIVE_OAK  \
TOWNSHIP_RANGE YEAR                                         
T01N R03E      2014                              0.000137   
               2015                              0.000137   
               2016                              0.000137   
               2017                              0.000137   
               2018                              0.000137   
...                                                   ...   
T32S R30E      2016                              0.000000 

In [40]:
# Change the shape of the input array to (number of Township-Ranges, 7 years (2014-2020), the number of features)
X_train = X_train_impute_df.values.reshape(len(X_train_impute_df.index.get_level_values(0).unique()), len(X_train_impute_df.index.get_level_values(1).unique()), X_train_impute_df.shape[1])
X_test = X_test_impute_df.values.reshape(len(X_test_impute_df.index.get_level_values(0).unique()), len(X_test_impute_df.index.get_level_values(1).unique()), X_test_impute_df.shape[1])

In [41]:
print("="*100)
print("Checking the train, validation and test input (X) datasets sizes:")
print(f"Size of the X_train dataset: {X_train.shape}")
#print(f"Size of the X_val dataset: {X_val.shape}")
print(f"Size of the X_test dataset: {X_test.shape}")
print("="*100)
print("Checking the train, validation and test output (y) datasets sizes:")
print(f"Size of the y_train dataset: {y_train.shape}")
#print(f"Size of the y_val dataset: {y_val_df.shape}")
print(f"Size of the y_test dataset: {y_test.shape}")

Checking the train, validation and test input (X) datasets sizes:
Size of the X_train dataset: (382, 7, 81)
Size of the X_test dataset: (96, 7, 81)
Checking the train, validation and test output (y) datasets sizes:
Size of the y_train dataset: (382, 1)
Size of the y_test dataset: (96, 1)


In [42]:
results_df = y_test_df[["GSE_GWE"]].copy()

In [43]:
nb_features = len(X_train_impute_df.columns)

adam_optimizer = Adam(learning_rate=hyper_parameters["learning_rate"])
rms_optimizer = RMSprop(learning_rate=hyper_parameters["learning_rate"])
adamax_optimizer = Adamax(learning_rate=hyper_parameters["learning_rate"])

In [44]:
def evaluate_forecast(y_test_inverse, yhat_inverse):
    mse_ = keras.metrics.MeanSquaredError()
    mae_ = keras.metrics.MeanAbsoluteError()
    rmse_ = keras.metrics.RootMeanSquaredError()
    mae = mae_(y_test_inverse,yhat_inverse)
    print('mae:', mae)
    mse = mse_(y_test_inverse,yhat_inverse)
    print('mse:', mse)
    rmse = rmse_(y_test_inverse,yhat_inverse)
    print('rmse:', rmse)
    return mae, mse, rmse

In [45]:
hyper_parameters = {
    "validation_split": 0.05,
    "learning_rate": 0.001,
    "batch_size": 32,
    "epochs": 100,
    "lstm_units": 200,
    "output_activation": "linear",
}

model1 = Sequential()
model1.add(LSTM(hyper_parameters["lstm_units"], input_shape=(7, nb_features)))
model1.add(Dense(1, activation=hyper_parameters["output_activation"]))
model1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 200)               225600    
                                                                 
 dense_6 (Dense)             (None, 1)                 201       
                                                                 
Total params: 225,801
Trainable params: 225,801
Non-trainable params: 0
_________________________________________________________________


In [46]:
# Start experiment
run = neptune.init(
    project="mlienart/milestone2",
    api_token=neptune_key,
    name="Basic Model"
)
neptune_cbk = NeptuneCallback(run=run, base_namespace='metrics')
hyper_parameters["optimizer"] = "RMSprop"
run['hyper-parameters'] = hyper_parameters

model1.compile(loss="mse", optimizer=rms_optimizer)
history = model1.fit(X_train, y_train,
                     validation_split=hyper_parameters["validation_split"],
                     batch_size=hyper_parameters["batch_size"],
                     epochs=hyper_parameters["epochs"],
                     shuffle=True,
                     callbacks=[neptune_cbk])
yhat = model1.predict(X_test, verbose=0)
yhat_inverse = scaler.inverse_transform(yhat)
y_test_inverse = scaler.inverse_transform(y_test)
results_df["experiment_1_prediction"] = yhat_inverse
mae, mse, rmse = evaluate_forecast(y_test_inverse, yhat_inverse)
run["eval/mae"] = mae
run["eval/mse"] = mse
run["eval/rmse"] = rmse
run.stop()

https://app.neptune.ai/mlienart/milestone2/e/MIL06KDIEQ-10
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Epoch 1/100
12/12 [==============================] - 2s 47ms/step - loss: 0.0220 - val_loss: 0.0054
Epoch 2/100
12/12 [==============================] - 0s 14ms/step - loss: 0.0086 - val_loss: 0.0073
Epoch 3/100
12/12 [==============================] - 0s 14ms/step - loss: 0.0059 - val_loss: 0.0157
Epoch 4/100
12/12 [==============================] - 0s 14ms/step - loss: 0.0042 - val_loss: 0.0081
Epoch 5/100
12/12 [==============================] - 0s 14ms/step - loss: 0.0061 - val_loss: 0.0068
Epoch 6/100
12/12 [==============================] - 0s 14ms/step - loss: 0.0049 - val_loss: 0.0345
Epoch 7/100
12/12 [==============================] - 0s 14ms/step - loss: 0.0045 - val_loss: 0.0056
Epoch 8/100
12/12 [====

In [47]:
hyper_parameters = {
    "validation_split": 0.05,
    "learning_rate": 0.001,
    "batch_size": 32,
    "epochs": 100,
    "lstm_units": 200,
    "dense_units": 20,
    "dropout": 0.2,
    "output_activation": "linear",
}

model2 = Sequential()
model2.add(Bidirectional(LSTM(hyper_parameters["lstm_units"]), input_shape=(7, nb_features)))
model2.add(Dense(hyper_parameters["dense_units"], activation="tanh"))
model2.add(Dropout(hyper_parameters["dropout"]))
model2.add(Dense(1, activation=hyper_parameters["output_activation"]))
model2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 400)              451200    
 nal)                                                            
                                                                 
 dense_7 (Dense)             (None, 20)                8020      
                                                                 
 dropout_2 (Dropout)         (None, 20)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 21        
                                                                 
Total params: 459,241
Trainable params: 459,241
Non-trainable params: 0
_________________________________________________________________


In [48]:
run = neptune.init(
    project="mlienart/milestone2",
    api_token=neptune_key,
    name="Advanced Model 1"
)
neptune_cbk = NeptuneCallback(run=run, base_namespace='metrics')
hyper_parameters["optimizer"] = "Adam"
run['hyper-parameters'] = hyper_parameters

model2.compile(loss="mse", optimizer=adam_optimizer)
history = model2.fit(X_train, y_train,
                     validation_split=hyper_parameters["validation_split"],
                     batch_size=hyper_parameters["batch_size"],
                     epochs=hyper_parameters["epochs"],
                     shuffle=True,
                     callbacks=[neptune_cbk])
yhat = model2.predict(X_test, verbose=0)
yhat_inverse = scaler.inverse_transform(yhat)
y_test_inverse = scaler.inverse_transform(y_test)
results_df["experiment_2_prediction"] = yhat_inverse
evaluate_forecast(y_test_inverse, yhat_inverse)
mae, mse, rmse = evaluate_forecast(y_test_inverse, yhat_inverse)
run["eval/mae"] = mae
run["eval/mse"] = mse
run["eval/rmse"] = rmse
run.stop()

https://app.neptune.ai/mlienart/milestone2/e/MIL06KDIEQ-11
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Epoch 1/100
12/12 [==============================] - 4s 132ms/step - loss: 0.0266 - val_loss: 0.0173
Epoch 2/100
12/12 [==============================] - 1s 85ms/step - loss: 0.0108 - val_loss: 0.0054
Epoch 3/100
12/12 [==============================] - 1s 83ms/step - loss: 0.0071 - val_loss: 0.0053
Epoch 4/100
12/12 [==============================] - 1s 88ms/step - loss: 0.0055 - val_loss: 0.0099
Epoch 5/100
12/12 [==============================] - 1s 87ms/step - loss: 0.0044 - val_loss: 0.0073
Epoch 6/100
12/12 [==============================] - 1s 87ms/step - loss: 0.0055 - val_loss: 0.0055
Epoch 7/100
12/12 [==============================] - 1s 87ms/step - loss: 0.0055 - val_loss: 0.0040
Epoch 8/100
12/12 [===

In [49]:
run = neptune.init(
    project="mlienart/milestone2",
    api_token=neptune_key,
    name="Advanced Model 1"
)
neptune_cbk = NeptuneCallback(run=run, base_namespace='metrics')
hyper_parameters["optimizer"] = "RMSprop"
run['hyper-parameters'] = hyper_parameters

model2.compile(loss="mse", optimizer=rms_optimizer)
history = model2.fit(X_train, y_train,
                     validation_split=hyper_parameters["validation_split"],
                     batch_size=hyper_parameters["batch_size"],
                     epochs=hyper_parameters["epochs"],
                     shuffle=True,
                     callbacks=[neptune_cbk])
yhat = model2.predict(X_test, verbose=0)
yhat_inverse = scaler.inverse_transform(yhat)
y_test_inverse = scaler.inverse_transform(y_test)
results_df["experiment_3_prediction"] = yhat_inverse
evaluate_forecast(y_test_inverse, yhat_inverse)
mae, mse, rmse = evaluate_forecast(y_test_inverse, yhat_inverse)
run["eval/mae"] = mae
run["eval/mse"] = mse
run["eval/rmse"] = rmse
run.stop()

https://app.neptune.ai/mlienart/milestone2/e/MIL06KDIEQ-12
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Epoch 1/100
12/12 [==============================] - 4s 146ms/step - loss: 0.0069 - val_loss: 0.0041
Epoch 2/100
12/12 [==============================] - 1s 83ms/step - loss: 0.0018 - val_loss: 0.0161
Epoch 3/100
12/12 [==============================] - 1s 85ms/step - loss: 0.0031 - val_loss: 0.0051
Epoch 4/100
12/12 [==============================] - 1s 87ms/step - loss: 0.0022 - val_loss: 0.0037
Epoch 5/100
12/12 [==============================] - 1s 85ms/step - loss: 0.0029 - val_loss: 0.0029
Epoch 6/100
12/12 [==============================] - 1s 86ms/step - loss: 0.0021 - val_loss: 0.0032
Epoch 7/100
12/12 [==============================] - 1s 88ms/step - loss: 0.0025 - val_loss: 0.0042
Epoch 8/100
12/12 [===

In [50]:
run = neptune.init(
    project="mlienart/milestone2",
    api_token=neptune_key,
    name="Advanced Model 1"
)
neptune_cbk = NeptuneCallback(run=run, base_namespace='metrics')
hyper_parameters["optimizer"] = "Adamx"
run['hyper-parameters'] = hyper_parameters

model2.compile(loss="mse", optimizer=adamax_optimizer)
history = model2.fit(X_train, y_train,
                     validation_split=hyper_parameters["validation_split"],
                     batch_size=hyper_parameters["batch_size"],
                     epochs=hyper_parameters["epochs"],
                     shuffle=True,
                     callbacks=[neptune_cbk])
yhat = model2.predict(X_test, verbose=0)
yhat_inverse = scaler.inverse_transform(yhat)
y_test_inverse = scaler.inverse_transform(y_test)
results_df["experiment_4_prediction"] = yhat_inverse
evaluate_forecast(y_test_inverse, yhat_inverse)
mae, mse, rmse = evaluate_forecast(y_test_inverse, yhat_inverse)
run["eval/mae"] = mae
run["eval/mse"] = mse
run["eval/rmse"] = rmse
run.stop()

https://app.neptune.ai/mlienart/milestone2/e/MIL06KDIEQ-13
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Epoch 1/100
12/12 [==============================] - 4s 145ms/step - loss: 9.6141e-04 - val_loss: 0.0029
Epoch 2/100
12/12 [==============================] - 1s 93ms/step - loss: 8.8754e-04 - val_loss: 0.0033
Epoch 3/100
12/12 [==============================] - 1s 92ms/step - loss: 7.0335e-04 - val_loss: 0.0031
Epoch 4/100
12/12 [==============================] - 1s 92ms/step - loss: 6.1270e-04 - val_loss: 0.0029
Epoch 5/100
12/12 [==============================] - 1s 93ms/step - loss: 8.2415e-04 - val_loss: 0.0030
Epoch 6/100
12/12 [==============================] - 1s 97ms/step - loss: 8.9607e-04 - val_loss: 0.0032
Epoch 7/100
12/12 [==============================] - 1s 96ms/step - loss: 6.4623e-04 - val_loss: 0

In [51]:
results_df

,,GSE_GWE,experiment_1_prediction,experiment_2_prediction,experiment_3_prediction,experiment_4_prediction
TOWNSHIP_RANGE,YEAR,,,,,
T01N R02E,2021,53.193636,65.490639,64.366951,54.697304,57.295902
T01N R11E,2021,107.955000,146.327942,174.259918,133.298782,149.292389
T01S R03E,2021,24.494538,32.694504,37.510136,38.773586,34.343391
T01S R07E,2021,38.644000,36.654675,39.726437,35.594738,43.306934
T01S R10E,2021,113.651250,115.802238,122.838219,111.689659,110.948303
...,...,...,...,...,...,...
T31S R26E,2021,173.915909,204.272202,230.257080,184.224365,201.201935
T31S R31E,2021,403.900000,350.634827,420.025665,392.852051,360.723633
T32S R22E,2021,160.340000,218.650299,273.818573,254.131851,230.082672
